In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from datetime import datetime 
import glob
from importlib import reload
import extract_features
import os
import numpy as np
import json
import pickle

json_path_project = 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\BATTERY\\'
json_path_onedrive = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\'

In [ ]:
colors = {
    'nightblue': '#332288',
    'darkgreen': '#117733',
    'turquoise': '#44AA99',
    'lightblue': '#88CCEE',
    'sand': '#DDCC77',
    'softred': '#CC6677',
    'lila': '#AA4499',
    'purplered': '#882255'
}

%matplotlib qt
# Create a figure and axis
fig, ax = plt.subplots()

# Plot lines for each color
x = range(len(colors))
for i, (color_name, color_code) in enumerate(colors.items()):
    y = [i] * len(x)
    ax.plot(x, y, color=color_code, lw=5, label=color_code)

# Set the x-ticks and labels
ax.set_xticks(x)
ax.set_xticklabels(colors.keys())

# Set the legend
ax.legend()

# Display the plot
plt.show()

### 1. Importing all Json files from Directory, and extracting the features

In [36]:
json_path_Subject = 'Sub059\\TK'

all_json_files = glob.glob(os.path.join(json_path_project, json_path_Subject, '*.json'))
filtered_files = [file for file in all_json_files if not os.path.basename(file).startswith('MetaTable') and not os.path.basename(file).startswith('Sub')]
filtered_files

['S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\BATTERY\\Sub059\\TK\\Report_Json_Session_Report_20240206T153053.json',
 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\BATTERY\\Sub059\\TK\\Report_Json_Session_Report_20240206T153115.json',
 'S:\\AG\\AG-Bewegungsstoerungen-II\\LFP\\PROJECTS\\BATTERY\\Sub059\\TK\\Report_Json_Session_Report_20240206T153131.json']

In [ ]:
'''with open(os.path.join(json_path, 'Report_Json_Session_Report_20200605T115803.json')) as file:
            # Load the JSON data
            data = json.load(file)
'''

In [37]:
reload(extract_features)
SUBID = json_path_Subject.split('\\', 1)[0]
json_path = os.path.join(json_path_project, json_path_Subject)
extract_features.extract_MetaTable(json_path_Subject, json_path, SUBID, filtered_files)

Report_Json_Session_Report_20240206T153053.json: Done!
Report_Json_Session_Report_20240206T153115.json: Done!
Report_Json_Session_Report_20240206T153131.json: Done!


### 2. Merging all Dataframes together for each subject

In [ ]:
mySub = SUBID
All_MetaTables_ThisSub_path = os.path.join(json_path_project, mySub)

# Initialize an empty list to store the DataFrames
dfs = []

# Iterate through the files in the directory
for filename in os.listdir(All_MetaTables_ThisSub_path):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(All_MetaTables_ThisSub_path, filename)
        
        # Read the JSON file and append the DataFrame to the list
        df = pd.read_json(file_path)
        dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
All_MetaTables_ThisSub_Combined = pd.concat(dfs, ignore_index=True)
conversion_factor = 24 * 60 * 60
All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] = All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial'] / conversion_factor
All_MetaTables_ThisSub_Combined.sort_values('AccumulatedTherapyOnTimeSinceImplant_Initial', inplace=True)
All_MetaTables_ThisSub_Combined.tail()

In [ ]:
'''All_MetaTables_ThisSub_Combined = pd.read_json(os.path.join(
    'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls',
    'Sub005_MetaTable_All.json'
))

All_MetaTables_ThisSub_Combined.head()'''

#### 2.1 Curating the Merged Df: Duplicates

In [ ]:
duplicates = All_MetaTables_ThisSub_Combined.duplicated(subset=['SessionDate', 'AccumulatedTherapyOnTimeSinceImplant_Initial'], keep=False)
duplicate_indices = duplicates[duplicates].index.tolist()
duplicate_rows = All_MetaTables_ThisSub_Combined[duplicates]

duplicate_rows

In [ ]:
All_MetaTables_ThisSub_Combined.drop_duplicates(subset=['SessionDate', 'AccumulatedTherapyOnTimeSinceImplant_Initial'], keep='first', inplace=True)
All_MetaTables_ThisSub_Combined

In [ ]:
'''# Get the last row of 'BatteryPercentage' for each unique 'Con_Reason' value
last_battery_percentages = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['BatteryPercentage'].last()

# Calculate the sum of 'overallSensingDurSec' for each unique 'Con_Reason' value
sum_sensing_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['overallSensingDurSec'].sum()/60
sum_telemetry_duration = All_MetaTables_ThisSub_Combined.groupby('Con_Reason')['telemetry_durationSec'].sum()/60

# Iterate through unique 'Con_Reason' values
for con_reason in All_MetaTables_ThisSub_Combined['Con_Reason'].unique():
    print(f"Last Battery Percentage for {con_reason}: {last_battery_percentages[con_reason]}")
    print(f"Sum of overallSensingDurSec for {con_reason}: {sum_sensing_duration[con_reason]}")
    print(f"Sum of telemetry_durationSec for {con_reason}: {sum_telemetry_duration[con_reason]}")
    print()'''

#### 2.2 Curating the Merged Df: Determine Follow-Up

In [ ]:
'''%matplotlib qt
plt.scatter(All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'], All_MetaTables_ThisSub_Combined['BatteryPercentage'])
plt.show()'''

In [ ]:
'''postOp_cutoff = 50
fu3m_cutoff = 120
fu12_cutoff = 360

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < postOp_cutoff, 'FollowUp'] = 'FU0M'

All_MetaTables_ThisSub_Combined.loc[
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > postOp_cutoff) &
    (All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] < fu3m_cutoff), 'FollowUp'] = 'FU3M'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > fu3m_cutoff, 'FollowUp'] = 'FU12M'

All_MetaTables_ThisSub_Combined.loc[
All_MetaTables_ThisSub_Combined['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'] > fu12_cutoff, 'FollowUp'] = 'LongTerm'

All_MetaTables_ThisSub_Combined.loc[
    All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Beelitz', 'FollowUp'] = 'Beelitz'

# Calculate mean of 'overallSensingDurSec' for rows where 'Con_Reason' is 'Ambulant'
mean_dur = All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant', 'overallSensingDurSec'].mean()
mean_dur1 = All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant', 'overallSensingDurSec'].mean()

# Update 'Con_Reason' column based on the mean value
All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant', 'Con_Reason'] = \
    All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'Ambulant'].apply(lambda row: 'Forschung' if row['overallSensingDurSec'] >= 10 else 'Ward', axis=1)

All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant', 'Con_Reason'] = \
    All_MetaTables_ThisSub_Combined.loc[All_MetaTables_ThisSub_Combined['Con_Reason'] == 'TAmbulant'].apply(lambda row: 'Forschung' if row['overallSensingDurSec'] >= 10 else 'Ward', axis=1)


#All_MetaTables_ThisSub_Combined['Con_Reason'] = All_MetaTables_ThisSub_Combined['Con_Reason'].apply(lambda x: 'Forschung' if x == 'True' else 'Ward')
All_MetaTables_ThisSub_Combined.head(20)'''

In [ ]:
#All_MetaTables_ThisSub_Combined.at[35, 'telemetry_durationSec'] = np.nan

'''[datetime.strptime(t, "%Y-%m-%dT%H:%M:%SZ") for t in All_MetaTables_ThisSub_Combined['SessionDate']]
arr_timestamps

import 3m timestamp
i_3mfu = np.argmin(abs(arr_timestamps - 3m_time))'''

## If all is Ok, save Dataframe Now!

In [ ]:
MetaTable_Results_Path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'

All_MetaTables_ThisSub_Combined.to_json(os.path.join(
    MetaTable_Results_Path,
    f'{mySub}_MetaTable_All.json'), 
    orient='records')

In [52]:
#### Excel MetaTables sort chronologically

MetaTable_Results_Path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls\\Done'

# Get the list of files in the directory
file_list = [file for file in os.listdir(MetaTable_Results_Path) if file.endswith('MetaTable_All.xlsx')]

# Iterate through the Excel files
for file in file_list:
    file_path = os.path.join(MetaTable_Results_Path, file)

    # Read the Excel file into a DataFrame
    df = pd.read_excel(file_path)

    # Add the 'Programming' column based on the condition
    df['Programming'] = df['Con_Reason'].apply(lambda x: 0 if x == 'Forschung' else 1)

    # Sort the DataFrame based on the 'AccumulatedTherapyOnTimeSinceImplant_Initial' column
    df_sorted = df.sort_values(by='AccumulatedTherapyOnTimeSinceImplant_Initial')

    # Write the sorted DataFrame back to the Excel file
    df_sorted.to_excel(file_path, index=False)

    print(f'Sorted and saved file: {file_path}')
    


Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub002_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub005_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub006_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub007_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub011_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Universitätsmedizin Berlin\BATTERY_LIFE\results\MetaTable_Alls\Done\Sub014_MetaTable_All.xlsx
Sorted and saved file: C:\Users\mathiopv\OneDrive - Charité - Un

### 3. Make average features for each follow-up and save them + Figures

In [58]:
path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls\\Done'
results_path_object = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\Avg_Features\\Avg_Features_Tbls'
figures_path_object = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\figures\\Overview\\'


file_list = []

# Loop through all files in the directory
for filename in os.listdir(path):
    if filename.endswith('MetaTable_All.xlsx') or filename.endswith('MetaTable_All.xlsx'):  # Filter Excel files
        file_list.append(filename)

for filename in file_list:

    All_MetaTables_ThisSub_Combined = pd.read_excel(os.path.join(path, filename))

    sub_id = filename[0:6]

    sums = pd.DataFrame(columns=['TimePoint', 'Telemetry_AllSec', 'TelemDurSumSecRes','TelemDurSumSecProgramming','SensDurSumSec', 'FirstBatVal', 'LastBatVal'])
    # Get the unique time points in the order they appear
    time_points_sorted = All_MetaTables_ThisSub_Combined['FollowUp'].unique()

    # Sort the time points based on their first occurrence in the DataFrame
    #time_points_sorted = sorted(time_points, key=lambda x: np.where(All_MetaTables_ThisSub_Combined['FollowUp'] == x)[0][0])

    # Get the corresponding y-values (last battery values)
    first_battery_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['BatteryPercentage'].first()
    last_battery_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['BatteryPercentage'].last()
    first_acc_therapy_values = All_MetaTables_ThisSub_Combined.groupby('FollowUp')['AccumulatedTherapyOnTimeSinceImplant_Initial_Days'].first()
    #y_values = [last_battery_values[tp] if tp in last_battery_values.index else 0 for tp in time_points_sorted]


    for tp in time_points_sorted:
        subset = All_MetaTables_ThisSub_Combined[All_MetaTables_ThisSub_Combined['FollowUp'] == tp]
        telemetry_all = subset['telemetry_durationSec'].sum()
        telemetry_sumRes = subset.loc[subset['Programming'] == 0, 'telemetry_durationSec'].sum()
        telemetry_sumProgramming = subset.loc[(subset['Programming'] == 1), 'telemetry_durationSec'].sum()
        sensing_sum = subset['overallSensingDurSec'].sum()
        first_battery_value = first_battery_values[tp] if tp in first_battery_values.index else None
        last_battery_value = last_battery_values[tp] if tp in last_battery_values.index else None
        first_acc_therapy_value = first_acc_therapy_values[tp] if tp in first_acc_therapy_values.index else None
        
        sums = sums.append({
            'TimePoint': tp,
            'Telemetry_AllSec': telemetry_all,
            'TelemDurSumSecRes': telemetry_sumRes,
            'TelemDurSumSecProgramming': telemetry_sumProgramming,
            'SensDurSumSec': sensing_sum,
            'FirstBatVal': first_battery_value,
            'LastBatVal': last_battery_value,
            'AccTimeSinceImplant_Days': first_acc_therapy_value
        }, ignore_index=True)
    
    sums.to_csv(os.path.join(
    results_path_object,
    f'{sub_id}_AvgFeatures.csv'
    ))
'''
    ####### PLOT IT #######
    tp_of_int = ['FU0M','Beelitz','FU3M','FU12M']

    # Filter the DataFrame to include only the specified follow-ups
    filtered_df = sums [sums ['TimePoint'].isin(tp_of_int)]

    %matplotlib inline

    fig, axs = plt.subplots(1,4, figsize=(18,4))

    axs[0].bar(filtered_df['TimePoint'], filtered_df['Telemetry_AllSec']/60)
    axs[0].bar(filtered_df['TimePoint'], filtered_df['TelemDurSumSecRes']/60, label = 'Research')
    axs[0].set_ylabel('Duration [min]')
    axs[0].set_title('Telemetry Duration')
    axs[0].legend()

    axs[1].bar(filtered_df['TimePoint'], filtered_df['SensDurSumSec']/60)
    axs[1].set_ylabel('Duration [min]')
    axs[1].set_title('Sensing Duration')

    subset = filtered_df[filtered_df['TimePoint'].isin(['FU0M', 'Beelitz', 'FU3M', 'FU12M'])]
    axs[2].bar(subset['TimePoint'], subset['TelemDurSumSecWard']/60)
    axs[2].set_ylabel('Duration [min]')
    axs[2].set_title('Ward Telemetry')

    axs[3].bar(filtered_df['TimePoint'], filtered_df['LastBatVal'])
    axs[3].set_ylabel('IPG Battery [%]')
    axs[3].set_title('Battery')

    plt.suptitle(sub_id)

    AvgValues_ThisSub_FigName = f'{sub_id}_Overview'

    fig.savefig(os.path.join(
        figures_path_object, AvgValues_ThisSub_FigName),
        dpi = 150)
'''

C:\Users\mathiopv\AppData\Local\Temp\ipykernel_9424\351189779.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sums = sums.append({
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_9424\351189779.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sums = sums.append({
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_9424\351189779.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sums = sums.append({
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_9424\351189779.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sums = sums.append({
C:\Users\mathiopv\AppData\Local\Temp\ipykernel_9424\351189779.py:43: FutureWarning: The frame.append

"\n    ####### PLOT IT #######\n    tp_of_int = ['FU0M','Beelitz','FU3M','FU12M']\n\n    # Filter the DataFrame to include only the specified follow-ups\n    filtered_df = sums [sums ['TimePoint'].isin(tp_of_int)]\n\n    %matplotlib inline\n\n    fig, axs = plt.subplots(1,4, figsize=(18,4))\n\n    axs[0].bar(filtered_df['TimePoint'], filtered_df['Telemetry_AllSec']/60)\n    axs[0].bar(filtered_df['TimePoint'], filtered_df['TelemDurSumSecRes']/60, label = 'Research')\n    axs[0].set_ylabel('Duration [min]')\n    axs[0].set_title('Telemetry Duration')\n    axs[0].legend()\n\n    axs[1].bar(filtered_df['TimePoint'], filtered_df['SensDurSumSec']/60)\n    axs[1].set_ylabel('Duration [min]')\n    axs[1].set_title('Sensing Duration')\n\n    subset = filtered_df[filtered_df['TimePoint'].isin(['FU0M', 'Beelitz', 'FU3M', 'FU12M'])]\n    axs[2].bar(subset['TimePoint'], subset['TelemDurSumSecWard']/60)\n    axs[2].set_ylabel('Duration [min]')\n    axs[2].set_title('Ward Telemetry')\n\n    axs[3].b

### 6. Find first 12mfu connection

In [ ]:
path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\MetaTable_Alls'

jsons_12mfu = pd.DataFrame(columns=['SubID', 'Con_reason','json_fileName'])

for filename in os.listdir(path):
    if filename.endswith('.xlsx'):  # Filter Excel files
    
        thisMetaTable = pd.read_excel(os.path.join(path, filename))
        
        sub_id = filename[0:6]
        fu12m_subset = thisMetaTable[thisMetaTable['FollowUp']=='FU12M']
        fu12m_subset = fu12m_subset.reset_index(drop=True)
        json_filename = fu12m_subset.loc[0,'json_fileName']
        con_reason = fu12m_subset.loc[0,'Con_Reason']
        
        jsons_12mfu = jsons_12mfu.append({
                'SubID': sub_id,
                'Con_reason': con_reason,
                'json_fileName': json_filename
            }, ignore_index=True)

In [ ]:
jsons_12mfu

In [ ]:
jsons_12mfu.to_excel(
    os.path.join(
        path,'Jsons_12mfu_StimPars_ToExtract4.xlsx'
    )
)

# Chronic Duplicates

#### 1. Identify and save the non-dups

In [ ]:
reload(extract_features)
SubID = 'Sub059'
directory_of_all = os.path.join(json_path_project, SubID)

reload(extract_features)
non_dups_chronics = extract_features.extract_chronic_nonDups(SubID, directory_of_all)

#### 2. Identify the date/time of the 12mfu 

In [ ]:
jsons_12mfu = pd.read_excel(
    os.path.join(
        json_path_onedrive,
        'results', 'Stim_pars','Done',
        'Jsons_12mfu_StimPars_ToExtract4.xlsx'))
#jsons_12mfu['SubID'] = jsons_12mfu['SubID'].apply(lambda x: x[0] if isinstance(x, tuple) and len(x) == 1 else x)

jsons_12mfu

In [ ]:
for index, row in jsons_12mfu.iterrows():
    
    this_subID = jsons_12mfu.loc[index,'SubID']
    
    MetaTable_path = os.path.join(
                json_path_onedrive,
                'results', 'MetaTable_Alls',
                f'{this_subID}_MetaTable_All.xlsx')
    
    MetaTable = pd.read_excel(MetaTable_path)
    
    t = jsons_12mfu.loc[index,'json_fileName']
    targeted_date = MetaTable.loc[MetaTable['json_fileName'] == t, 'SessionDate'].iloc[0] if t in MetaTable['json_fileName'].values else None

    jsons_12mfu.loc[index, 'DateTime'] = targeted_date


In [ ]:
jsons_12mfu.to_excel(os.path.join(json_path_onedrive,
                                  'results', 'Stim_pars', 'Done',
                                  'Jsons_12mfu_StimPars_ToExtract_Dates2.xlsx'),
                     index= None)

#### 3. Calculate amount of chronic until 12mfu

In [ ]:
jsons_12mfu = pd.read_excel(
    os.path.join(
        json_path_onedrive,
        'results', 'Stim_pars','Done',
        'Jsons_12mfu_StimPars_ToExtract_Dates.xlsx'))

jsons_12mfu.columns

In [ ]:
import pickle
from datetime import datetime

# Assuming you have already defined jsons_12mfu and json_path_project

chron_vals_dataframe = []

for index, row in jsons_12mfu.iterrows():
    this_subID = row['SubID']
    threshold_datetime_str = row['DateTime']
    
    threshold_datetime = datetime.strptime(threshold_datetime_str, '%Y-%m-%dT%H:%M:%SZ')

    pickle_file_path = os.path.join(json_path_project, 'Chronic_Sensing_NonDups', f'{this_subID}_NonDupsChronics.pkl')

    # Check if the pickle file exists
    if os.path.exists(pickle_file_path):
        with open(pickle_file_path, 'rb') as file:
            non_dups_chronics = pickle.load(file)
        
        non_dups_chronics = sorted(non_dups_chronics) 
        filtered_date_times = [dt for dt in non_dups_chronics if datetime.strptime(dt, '%Y-%m-%dT%H:%M:%SZ') <= threshold_datetime]
        
        this_chron_vals_dict = {
            'SubID': this_subID,
            'Chronic_Entries_12mfu': len(filtered_date_times),
            'Chronic_12mfu_Hours': len(filtered_date_times) / 6,
            'Chronic_12mfu_Days': (len(filtered_date_times) / 6) / 24,
        }
        
        chron_vals_dataframe.append(this_chron_vals_dict)

# Convert the list of dictionaries to a DataFrame
final_dataframe = pd.DataFrame(chron_vals_dataframe)

# Print the final DataFrame
print(final_dataframe)


In [ ]:
final_dataframe.to_csv(os.path.join(
    json_path_onedrive, 'results', 'Correlations', 'Chronic_nonDups_vals.csv'
), index = None)

## Demographics

In [ ]:
demos = pd.read_excel(os.path.join(
    json_path_onedrive, 'docs', 'Demos.xlsx'
), sheet_name= 'Demos_MS', index_col=None)

demos.rename(columns={'Percept_ID': 'SubID'}, inplace=True)

demos.head()

In [ ]:
#Full dataset
nfem = demos['Gender'].value_counts()[1]
mean_age = np.round(np.mean(demos['Age']), decimals = 2)
std_age = np.round(np.std(demos['Age']), decimals = 2)

print(f'Total N = {demos.shape[0]}')
print(f'N of female = {nfem}')
print(f'Mean age = {mean_age}')
print(f'Std age = {std_age}')

In [ ]:
#Only included patients
filtered_demos = demos[demos['Included'] == 1]
nfem1 = filtered_demos['Gender'].value_counts()[1]
mean_age1 = np.round(np.mean(filtered_demos['Age']), decimals = 2)
std_age1 = np.round(np.std(filtered_demos['Age']), decimals = 2)
Sens_n = filtered_demos[filtered_demos['Electrode']=='SenSight'].shape[0]
Beel_n = filtered_demos[filtered_demos['Beelitz']=='X'].shape[0]
Amb_n = filtered_demos[filtered_demos['Ambulant']=='X'].shape[0]

print(f'Total N = {filtered_demos.shape[0]}')
print(f'N of female = {nfem1}')
print(f'Mean age = {mean_age1}')
print(f'Std age = {std_age1}')
print(f'N of SenSight = {Sens_n}')
print(f'N of Beelitz = {Beel_n}')
print(f'N of Ambulanz = {Amb_n}')